# Decision trees
## Teori

- en massa if ... then ... else ... satser  
- constructing a binary tree over X  
- antalet nivåer i trädet kallas depths  
- vi försöker minimera entropi (Shannon enthropy)  
- entropi ökar med djupet, därför föredrar vi små träd (eller stubbar, djup 2, två noder)  
- träden är högst linjära, därför kan vi bryta upp dem i mindre delar  
    - då kan många (små) träd rösta för klassificering  
- entropi:
    - mäter mängden oordning (klassisk definition), mängden möjliga sätt jämfört med mängden utfall  
    - maximala mängden sätt som något kan fortgå på  
    - det finns mer information när vi har lägre entropi  
    - entropi kan vi säga mäter kvalitet genom brus, hög entropi = mycket brus  
    - det finns en direkt koppling mellan entropi och gini  

### Random forest  

- oftast bygger i random forests  
- choose random $x \epsilon \bold{X}$ and do a split  
- right angled decision boundaries  
- många träd röstar  
- denna metod behöver inte standardiseras eller normaliseras  
    - eftersom den bara gör splits
- däremot måste den hyperparameteroptimeras (HPO) 
    - max depth  
    - number of trees/estimators  
    - några fler beroende på  
- vi kan råka ut för kombinatorisk explosion med många hyperparametrar (HP)

## Code

### DT Regression

Det går att göra både klassificering och regression med träd. De har däremot väldigt hög bias. Resultaten för trädregression är dock inte så bra och därför ovanligt att de används för regression.  

Däremot kan du alltid köra den. Så om du inte skulle ha någon aning om vad du ska köra för modell så går det med träd. 

In [ ]:
y_test.hist()

Eftersom den ser geometrisk ut så kan vi inte göra någon linjär regression.  

[någonting] i stället för feature expansion kan vi göra kernel trick och komma undan billigare. 

In [ ]:
model_tree_reg.feature_importances_

Ger oss värde mellan 0-1 och visar på hur viktiga våra features är. T ex 0.58 säger att den kunde dra en linje där 58 % av datan är på ena sidan, resten på andra. 

In [ ]:
y_pred = model_tree_reg.predict(X_test)

mean_absolute_error(y_test, y_pred), np.sqrt(mean_squared_error(y_test, y_pred))

Vi kan inte räkna med att vi får bra stabilitet, vi får höga fel pga stora steg mot regressionsvärdena. 

### DT Classification

Om vi får låg relevans (feature importance = låg) så hamnar alla värden på ena sidan linljen. Så obalanserad data är DT inte bra för. Väldigt känsliga för overfit. 

# Random forest

Bagging innebär att vi tar flera slumpmässiga sticprov från samma träningsset och kollar medlet från alla dessa. Ganska likt CV men syftet är för att vi ska få en bättre förutsägelse. I klassifikationsproblem blir det alltså majoritetsröstning. Vi kan alltså ha samma data flera gånger men det spelar ingen roll för att vi vill ha många små träd och det är fortfarande slumpmässigt. Det leder i detta fallet alltså inte till data leakage.  



In [ ]:
df = pd.read_csv("../data/Heart.csv", index_col = 0)

I detta fallet är vår target variable AHD, en hjärtsjukdom. 

In [ ]:
param_grid = {"n_estimators": [100, 150, 200,
                               300], "criterion": ["gini", "entropy"], "max_features": ["auto","sqrt", "log2"]}

Om det tar för lång tid, testa att ta bort "auto". 

In [ ]:
clf = GridSearchCV(RandomForestClassifier(), param_grid,
                   cv=5, verbose=1, scoring="recall")

Vi vill inte ha falska negativa (säga att de är friska när de är sjuka) därför optimerar vi på "recall". 

Allmänt problem är att vi kommer få olika resultat när vi kör gång på gång för att de har hög varians. Efter HPO så har vi en bättre chans att få ett bättre resultat men det är inte garanterat. Det är däremot mycket dyrare oftast att också använda sig av HPO men beroende på resultat så vet vi inte om det är värt det eller inte.  

Det är pga att det är så mycket slump inblandat. RF ska inte användas i kritiska applikationer, enligt Raphael. Däremot i allmänhet bra att använda som en första modell och för enkla problem kan de vara väldigt kraftfulla. 

# Naive Bayes

Vi kan göra egna voting classifiers. 

In [ ]:
from sklearn.ensemble import VotingClassifier

Vi kan kombinera olika metoder. Du kan till och med träna dem innan och sen använda dem tillsammans. 

In [ ]:
vote_clf = VotingClassifier(
    [("rf", RandomForestClassifier()),
     ("svc", LinearSVC()),
     ("naive_complement", ComplementNB()),
     ("naive_bernoulli", BernoulliNB()),
     ("naive_multinomail", MultinomialNB())], voting="hard")

Om du misstänker hög varians så använd "hard" voting. 

generalisation power - stämmer bara när stickproven råkar vara lika

data augmentation - boosting - försöker göra något med den mindre datan när det är stor skillnad på storleken av klasserna. Basically boosta den obalanserade klassen. 